In [4]:
# import dependencies
import os
import io
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# from selenium import webdriver
# from lxml import html
import csv
from datetime import datetime
from pytz import timezone
import pandas as pd
import FoodTools

In [5]:
# url of ucsc college 9 & 10 dining hall
url = 'https://nutrition.sa.ucsc.edu/nutframe.asp?sName=UC+Santa+Cruz+Dining&locationNum=40&locationName=Colleges+Nine+%26+Ten+Dining+Hall&naFlag=1'

In [6]:
# scrap the menu data from the webpage
# since the menu data is cannot be scrapped from the html file directly
# we have to make additional requests to get the frame page contents 
with requests.Session() as session:
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for frame in soup.select("frameset frame"):
        frame_url = urljoin(url, frame["src"])
        response = session.get(frame_url)
        frame_soup = BeautifulSoup(response.content, 'html.parser') 
        # print(frame_soup.prettify())

In [7]:
# extract both the meal name and food name from html
meal_name = frame_soup.find_all('div', attrs={'class': 'menusampmeals'})
food_name = frame_soup.find_all('div', attrs={'class': 'menusamprecipes'})

In [8]:
# convert the name of the meal type in html form to a list form
meal_type = [item.string for item in meal_name]
print(meal_type)
print('Length: ', len(meal_type))

['Breakfast', 'Lunch', 'Dinner', 'Late Night']
Length:  4


In [9]:
# convert the food name in html form to a list form
food_list = [item.string for item in food_name]
print(food_list)
print('Length: ', len(food_list))

['Belgian Waffles', 'Cage Free Scrambled Eggs', 'Crispy Bacon', 'Eggbeaters Scramble', 'French Toast Sticks', 'Hard-boiled Cage Free Eggs', 'Natural BridgesTofu Scramble', 'Oatmeal Gluten-Free', 'Quinoa Fruit Salad', 'Tator Tots', 'Almond Allspice Coffecake', 'Carrot Raisin Muffin', 'Morning Glory Blueberry Granola Nut Muffin', 'Vegan Peanut Granola', 'Bell Peppers', 'Cage Free Eggs', 'Cage Free Omelette Bar', 'Cheddar Cheese', 'Feta Cheese', 'Fresh Organic Spinach', 'Fresh Tomato', 'Jalapeno', 'Onions', 'Oven Roasted Turkey', 'Sliced Olived', 'Smoked Ham', 'Green Pozole', 'Tomato Bisque Soup', 'Oven Roasted Chicken Thigh', 'Spice Crusted Pork Loin', 'Sweet & Sour Tofu', 'Sweet and Sour Shrimp', 'Cheese Pizza', 'Chicken Club Pizza', 'Roasted Yams', 'Steamed Rice', 'Sauteed Seasonal Vegetable', 'Rice Krispy Treats', 'Vegan Chocolate Cup Cakes', 'Cajun Bayou Bar', 'Cajun Rice with Red Beans', 'Chicken Honey Mustard', 'Chipotle BBQ Sauce', 'Corn Cobbettes', 'Jalapeno Corn Cakes', 'Sausage

In [10]:
# In case of dupliate food name, the list should be converted to a set
# sort
food_list = sorted(list(set(food_list)))
print(food_list)
print('Sorted length: ', len(food_list))

['Almond Allspice Coffecake', 'Bar Taqueria', 'Belgian Waffles', 'Bell Peppers', 'Boston Cream Cake', 'Butternut Squash with Chile Yogurt', 'Cage Free Eggs', 'Cage Free Omelette Bar', 'Cage Free Scrambled Eggs', 'Cajun Bayou Bar', 'Cajun Rice with Red Beans', 'Carrot Raisin Muffin', 'Charro Beans', 'Cheddar Cheese', 'Cheese Pizza', 'Cheese Quesadilla', 'Cheese Sauce', 'Chicken Club Pizza', 'Chicken Honey Mustard', 'Chicken Korma', 'Chipotle BBQ Sauce', 'Condiments', 'Corn Cobbettes', 'Crispy Bacon', 'Eggbeaters Scramble', 'Feta Cheese', 'French Toast Sticks', 'Fresh Organic Spinach', 'Fresh Tomato', 'Green Pozole', 'Hard-boiled Cage Free Eggs', 'Jalapeno', 'Jalapeno Corn Cakes', 'Mexican Rice', 'Morning Glory Blueberry Granola Nut Muffin', 'Nachos Bar', 'Natural BridgesTofu Scramble', 'Oatmeal Gluten-Free', 'Onion Rings', 'Onions', 'Orange Glazed Pork', 'Orange Glazed Vegan Chicken', 'Organic Brown Basmati Rice', 'Oven Roasted Chicken Thigh', 'Oven Roasted Turkey', 'Penne with Sun Drie

In [ ]:
# preference list, labeled by myself
# pref_1 = [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0 ]
# pref_2 = [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0]
# pref_3 = [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1]
# pref_4 = [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0]
# pref_5 = [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
pref = []
for i in food_list:
    print(i)
    preference = int(input())
    if preference != 0 and preference != 1:
        print('Incrorrect input')
        break
    pref.append(preference)
print(pref)
print('Preference length: ', len(pref))

Almond Allspice Coffecake
0
Bar Taqueria
0
Belgian Waffles
1
Bell Peppers
0
Boston Cream Cake
1
Butternut Squash with Chile Yogurt
0
Cage Free Eggs
0
Cage Free Omelette Bar
0
Cage Free Scrambled Eggs
1
Cajun Bayou Bar
0
Cajun Rice with Red Beans
0
Carrot Raisin Muffin
0
Charro Beans
0
Cheddar Cheese
0
Cheese Pizza
1
Cheese Quesadilla
0
Cheese Sauce
0
Chicken Club Pizza
1
Chicken Honey Mustard
0
Chicken Korma
0
Chipotle BBQ Sauce
1
Condiments
0
Corn Cobbettes
0
Crispy Bacon
1
Eggbeaters Scramble
0
Feta Cheese
0
French Toast Sticks
1
Fresh Organic Spinach
0
Fresh Tomato
1
Green Pozole
0
Hard-boiled Cage Free Eggs
0
Jalapeno
0
Jalapeno Corn Cakes
0
Mexican Rice
0
Morning Glory Blueberry Granola Nut Muffin
0
Nachos Bar
0
Natural BridgesTofu Scramble
0
Oatmeal Gluten-Free
0
Onion Rings
1
Onions
0
Orange Glazed Pork
1
Orange Glazed Vegan Chicken
0
Organic Brown Basmati Rice
0
Oven Roasted Chicken Thigh
0
Oven Roasted Turkey
0
Penne with Sun Dried Tomato, Spinach and Portabella
0
Pork Carnita

In [11]:
# timezone setting
us_pacific = timezone('US/Pacific')
time = datetime.now(us_pacific)
us_time = time.strftime('%Y-%m-%d')
print('Current US/Pacific time: ', us_time)

Current US/Pacific time:  2018-09-26


In [12]:
# write the food list to a csv file
path = '/Users/7w0r4ng3s/Desktop/menu_scraping/data/{}.csv'.format(us_time)
print('Current path: ', path)
path_2 = '/Users/7w0r4ng3s/Desktop/menu_scraping/data/'

Current path:  /Users/7w0r4ng3s/Desktop/menu_scraping/data/2018-09-26.csv


In [13]:
def write_data():
    with open(path, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        for key, val in zip(food_list, pref):
            writer.writerow([key, val])
    print('write_csv: COMPLETED')
    # add index and column names
    df = pd.read_csv(path, names=['food', 'pref'])
    df.index.names = ['index']
    df.to_csv(path)
    print('add_column_name: COMPLETED')
    
write_data()

write_csv: COMPLETED
add_column_name: COMPLETED


In [14]:
# def merge_data():
#     # TODO: Figure out a way to get rid of the unnamed: 0 column
#     # TODO: Modify merge_data() so that new data can be append to data.csv
#     files = [f for f in os.listdir('.') if os.path.isfile(f)]

#     merged = []

#     for f in files:
#         filename, ext = os.path.splitext(f)
#         if ext == '.csv':
#             read = pd.read_csv(f)
#             merged.append(read)

#     result = pd.concat(merged)
#     result.to_csv('data.csv')
    
# merge_data()

In [15]:
def append_data():
    df1 = pd.read_csv('Data.csv', index_col='index')
    df1.index.names = ['index']
    df2 = pd.read_csv(path, index_col='index')
    df3 = df1.append(df2).reset_index()
    df3 = df3.drop('index', 1).sort_values('food').reset_index().drop('index', 1)
    df3.index.names = ['index']
    df3.to_csv('Data.csv')
    
append_data()

In [16]:
FoodTools.search()

Search food: char
Do you mean:
1 Char Siu Sauce
2 Char Siu Seitan
3 Charro Beans
4 Garlic Rainbow Chard

Enter the index: 2
--------------------------------------------------------------------------------
Out of 1 time(s), you like [Char Siu Seitan] 1 time(s).
--------------------------------------------------------------------------------


1.0